In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import nltk
import string
from nltk.corpus import stopwords
from nltk. tokenize import word_tokenize 
from nltk.stem import SnowballStemmer 
nltk.download ('punkt')
from sklearn.pipeline import Pipeline 
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction. text import TfidfVectorizer
from sklearn.metrics import precision_score, recall_score, precision_recall_curve 
from matplotlib import pyplot as plt
from sklearn.metrics import precision_recall_curve
import numpy as np 
from sklearn.model_selection import GridSearchCV
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/andreyvasilyev/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/andreyvasilyev/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

сохраняем тренировочный датасет в переменную df

In [2]:
df=pd.read_csv('train.csv')

просматриваем датасет

In [3]:
print(df)

            ID                                               text  sentiment
0        21098  .с.,и спросил его:  о Посланник Аллаха!Ты пори...          1
1        21099  Роднее всех родных Попала я в ГКБ №8 еще в дек...          1
2        21100  Непорядочное отношение к своим работникам Рабо...          2
3        21101  ). Отсутствуют нормативы, Госты и прочее, что ...          1
4        21102             У меня машина в руках 5 лет и это п...          1
...        ...                                                ...        ...
189886  210984  Мой юбилей я отмечал в ресторане " Астория " ....          2
189887  210985  Отлично встретили, разместили в роскошном номе...          1
189888  210986  Была в Васаби на ст. метро Сенная .  Во первых...          0
189889  210987  Ребята не стоит смотреть этот фильм. Вы молоды...          0
189890  210988  Про махание руками нигде  не нашел.  Почему се...          1

[189891 rows x 3 columns]


In [4]:
df.shape

(189891, 3)

разбиение на токены

In [5]:

sentence_example = df. iloc [1] ["text" ]

tokens = word_tokenize(sentence_example, language="russian")

tokens_without_punctuation = [i for i in tokens if i not in string. punctuation]

russian_stop_words = stopwords. words ("russian")

tokens_without_stop_words_and_punctuation = [i for i in tokens_without_punctuation if i not in russian_stop_words]

snowball = SnowballStemmer (language="russian")

stemmed_tokens = [snowball.stem(i) for i in tokens_without_stop_words_and_punctuation]

просмотр корректности работы

In [6]:
print(f"Исходный текст: {sentence_example}")
print("———————-")
print (f"Токены: {tokens}")
print("————————-")
print (f"Токены без пунктуации: {tokens_without_punctuation}")
print("—————————")
print(f"Токены без пунктуации и стоп слов: {tokens_without_stop_words_and_punctuation}")
print ("—————")
print (f"Токены после стемминга: {stemmed_tokens}")
print("————")

Исходный текст: Роднее всех родных Попала я в ГКБ №8 еще в декабре 2002 г. Ехать в больницу очень боялась, но делать нечего, рожать надо. О ГКБ № 8 слышала много хорошего. Когда приехала в приемное отделение, мои колени тряслись от страха, но меня там встретили очень вежливо. Осмотр врача был аккуратным и осторожным, во всем чувствовалась забота и внимание. После того, как меня определили в палату, моим врачом стала Попова Любовь Васильевна. Я благодарна судьбе за то, что она свела меня с таким прекрасным человеком. Она огромнейшей души человек и супер-врач. Внимательна, тактична, отвечала на все, порой даже глупые вопросы. Всегда может подбодрить, улыбнуться. Расскажет зачем и какие препараты нам назначают, а также зачем и какие анализы мы сдаем. С ней я себя чувствовала как "за каменной стеной". К тому же у нас собралась веселая палата. На этаже есть телевизор, работае с 17 до 22 часов. Вообщем-то этого вполне хватало, т. к. у каждого были свои процедуры, капельницы, уколы и т. д. По

написание функции преобразования в токены для дальнейшего использования

In [7]:
snowball = SnowballStemmer (language="russian" )
russian_stop_words = stopwords.words ("russian" )
def tokenize_sentence(sentence: str, remove_stop_words: bool = True):
    tokens = word_tokenize(sentence, language="russian" )
    tokens = [i for i in tokens if i not in string. punctuation]
    if remove_stop_words:
        tokens = [i for i in tokens if i not in russian_stop_words]
    tokens = [snowball.stem(i) for i in tokens]
    return tokens

векторизация

In [8]:
vectorizer = TfidfVectorizer(tokenizer=lambda x: tokenize_sentence(x, remove_stop_words=True),token_pattern=None)

обучение векторайзера

In [9]:
features = vectorizer. fit_transform(df["text"] )

обучение модели логистической регрессии

In [11]:
model = LogisticRegression (random_state=0,max_iter=200000)
model.fit(features, df["sentiment"])

LogisticRegression(max_iter=200000, random_state=0)

проверка модели

In [12]:
model.predict (features[0])

array([1])

преобразование модели

In [14]:
model_pipeline = Pipeline ([
    ("vectorizer", TfidfVectorizer (tokenizer=lambda x: tokenize_sentence(x, remove_stop_words=True),token_pattern=None)), 
    ("model", LogisticRegression(random_state=0,max_iter=200000))
])

In [15]:
model_pipeline. fit(df["text"], df["sentiment"])

Pipeline(steps=[('vectorizer',
                 TfidfVectorizer(token_pattern=None,
                                 tokenizer=<function <lambda> at 0x104b080e0>)),
                ('model', LogisticRegression(max_iter=200000, random_state=0))])

сохранение тестового датасета с текстами в переменную

In [16]:
df_test=pd.read_csv('test.csv')

просмотр

In [17]:
df_test

,ID,text
0,0,Развода на деньги нет Наблюдаюсь в Лайфклиник ...
1,1,Отель выбрали потому что рядом со стадионом. О...
2,2,"Вылечили Гноился с рождения глазик, в поликлин..."
3,3,Хорошее расположение.С вокзала дошли пешком.Но...
4,4,"Отличное месторасположение,прекрасный вид,особ..."
...,...,...
21093,21093,Несколько лет назад муж останавливался в этом ...
21094,21094,Спасли от боли После родов у меня появились бо...
21095,21095,з ролика понятно одно - девушка- наблюдатель ...
21096,21096,"Остались всем довольны, дружелюбный персонал, ..."


сохранение тестового датасета с значениями в переменную

In [18]:
df_test_sub=pd.read_csv('sample_submission.csv')

просмотр

In [19]:
df_test_sub

,ID,sentiment
0,0,0
1,1,2
2,2,2
3,3,2
4,4,1
...,...,...
21093,21093,0
21094,21094,0
21095,21095,0
21096,21096,0


тест модели

In [20]:
model_pipeline.predict( [df_test['text'][21097]] )

array([1])

In [21]:
model_pipeline.predict( [df_test['text'][4]] )

array([1])

предсказание

In [22]:
a=[]
for i in range (len(df_test['text'])):
    a.append(model_pipeline.predict([df_test['text'][i]] )[0])
print(a)

[2, 0, 1, 0, 1, 1, 2, 0, 1, 0, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 0, 1, 2, 2, 1, 2, 2, 2, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 0, 2, 2, 1, 1, 1, 2, 0, 0, 1, 1, 0, 1, 0, 2, 0, 0, 0, 1, 2, 1, 2, 1, 2, 1, 2, 1, 0, 2, 1, 2, 0, 0, 0, 0, 2, 0, 2, 0, 0, 2, 1, 1, 1, 0, 1, 1, 1, 1, 2, 1, 2, 1, 2, 0, 2, 1, 2, 2, 1, 0, 2, 1, 0, 0, 1, 1, 1, 2, 1, 2, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 2, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 2, 1, 2, 0, 2, 1, 0, 1, 2, 2, 1, 2, 0, 2, 1, 0, 2, 2, 0, 1, 1, 0, 0, 1, 2, 2, 1, 2, 2, 1, 2, 2, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 2, 2, 2, 0, 0, 1, 1, 1, 2, 0, 0, 1, 0, 1, 1, 2, 2, 2, 1, 0, 1, 1, 1, 2, 1, 1, 1, 1, 2, 0, 0, 2, 0, 1, 1, 2, 0, 0, 0, 2, 0, 1, 1, 1, 1, 1, 2, 2, 2, 1, 0, 2, 1, 2, 2, 0, 1, 0, 1, 1, 1, 1, 0, 2, 1, 1, 1, 1, 2, 1, 1, 1, 0, 0, 2, 2, 2, 1, 1, 2, 1, 0, 0, 2, 2, 1, 0, 0, 1, 0, 1, 1, 2, 2, 1, 0, 2, 2, 1, 1, 1, 2, 2, 1, 2, 2, 0, 2, 1, 1, 2, 0, 1, 1, 2, 1, 0, 2, 1, 2, 1, 0, 2, 1, 0, 1, 1, 0, 1, 1, 1, 1, 2, 1, 0, 1, 1, 1, 0, 1, 0, 1, 2, 0, 1, 0, 1, 2, 

In [23]:
df_test2=pd.read_csv('test.csv')

добавление предсказаний в датасет

In [24]:
df_test2["sentiment"]=a
print(df_test2)

          ID                                               text  sentiment
0          0  Развода на деньги нет Наблюдаюсь в Лайфклиник ...          2
1          1  Отель выбрали потому что рядом со стадионом. О...          0
2          2  Вылечили Гноился с рождения глазик, в поликлин...          1
3          3  Хорошее расположение.С вокзала дошли пешком.Но...          0
4          4  Отличное месторасположение,прекрасный вид,особ...          1
...      ...                                                ...        ...
21093  21093  Несколько лет назад муж останавливался в этом ...          0
21094  21094  Спасли от боли После родов у меня появились бо...          1
21095  21095  з ролика понятно одно - девушка- наблюдатель  ...          0
21096  21096  Остались всем довольны, дружелюбный персонал, ...          1
21097  21097  Спасибо огромное за сыночка Хочу выразить огро...          1

[21098 rows x 3 columns]


In [28]:
precision_score(y_true=df_test_sub["sentiment"], y_pred=model_pipeline. predict(df_test["text" ]), average='micro')

0.3360034126457484

In [29]:
recall_score(y_true=df_test_sub["sentiment"], y_pred=model_pipeline. predict(df_test["text" ]), average='micro')

0.3360034126457484

In [69]:
df_test2.to_csv('df_test.csv', encoding='utf-8')